In [11]:
import pandas as pd
import requests
import json

In [12]:
df = pd.read_csv('./stats/player/dlillard_2021-22_off_stats.csv')

print(df.shape)
df.head()

(189, 27)


,SEASON_ID,OFF_PLAYER_ID,OFF_PLAYER_NAME,DEF_PLAYER_ID,DEF_PLAYER_NAME,GP,MATCHUP_MIN,PARTIAL_POSS,PLAYER_PTS,TEAM_PTS,...,MATCHUP_FG3A,MATCHUP_FG3_PCT,HELP_BLK,HELP_FGM,HELP_FGA,HELP_FG_PERC,MATCHUP_FTM,MATCHUP_FTA,SFL,MATCHUP_TIME_SEC
0,22021,203081,Damian Lillard,1628969,Mikal Bridges,3,24:22,126.3,28,132,...,12,0.333,0,0,0,0,0,0,0,1461.9
1,22021,203081,Damian Lillard,1628415,Dillon Brooks,2,19:21,94.3,20,88,...,10,0.400,0,0,0,0,4,5,3,1161.5
2,22021,203081,Damian Lillard,1630178,Tyrese Maxey,2,13:19,65.1,23,83,...,11,0.455,0,0,0,0,2,2,0,798.7
3,22021,203081,Damian Lillard,202339,Eric Bledsoe,3,10:10,58.7,18,62,...,6,0.667,0,0,0,0,2,2,1,609.8
4,22021,203081,Damian Lillard,202340,Avery Bradley,2,10:31,55.6,7,46,...,7,0.143,0,0,0,0,4,4,0,630.9


**Convert totals to per 100 possessions**

Most limited sample size, convert each to per possession and then multiply by 100 (more standard to measure in per 100 possessions rather than per possession, as generally nba teams average [100 possessions per game](https://www.teamrankings.com/nba/stat/possessions-per-game)).

In [13]:
MIN_MATCHUP_MINS = 5

In [26]:
off_df = df.copy()
off_df = off_df[off_df['MATCHUP_TIME_SEC'] > MIN_MATCHUP_MINS * 60] # Must have played more than x minutes
off_df.drop(columns=['TEAM_PTS', 'MATCHUP_FG_PCT', 'MATCHUP_FG3_PCT'], inplace=True)

def per_100_poss(x):
    return x / off_df['PARTIAL_POSS'] * 100

# Set stats to per 100 possessions
off_df = off_df.apply(lambda x: per_100_poss(x) if x.name not in off_df.columns[0:8] else x)

off_df.sort_values('DEF_PLAYER_NAME', ascending=True, inplace=True)
print(off_df.shape)
off_df.head()

(21, 24)


,SEASON_ID,OFF_PLAYER_ID,OFF_PLAYER_NAME,DEF_PLAYER_ID,DEF_PLAYER_NAME,GP,MATCHUP_MIN,PARTIAL_POSS,PLAYER_PTS,MATCHUP_AST,...,MATCHUP_FG3M,MATCHUP_FG3A,HELP_BLK,HELP_FGM,HELP_FGA,HELP_FG_PERC,MATCHUP_FTM,MATCHUP_FTA,SFL,MATCHUP_TIME_SEC
4,22021,203081,Damian Lillard,202340,Avery Bradley,2,10:31,55.6,12.589928,7.194245,...,1.798561,12.589928,0.0,0.0,0.0,0.0,7.194245,7.194245,0.000000,1134.712230
19,22021,203081,Damian Lillard,1629012,Collin Sexton,1,5:07,26.9,11.152416,7.434944,...,0.000000,7.434944,0.0,0.0,0.0,0.0,3.717472,3.717472,3.717472,1140.520446
5,22021,203081,Damian Lillard,1630558,Davion Mitchell,2,9:16,47.5,23.157895,6.315789,...,0.000000,14.736842,0.0,0.0,0.0,0.0,18.947368,18.947368,4.210526,1169.894737
16,22021,203081,Damian Lillard,1628368,De'Aaron Fox,2,5:01,32.4,27.777778,9.259259,...,3.086420,12.345679,0.0,0.0,0.0,0.0,6.172840,9.259259,3.086420,929.938272
15,22021,203081,Damian Lillard,1629001,De'Anthony Melton,3,7:03,36.3,11.019284,11.019284,...,0.000000,2.754821,0.0,0.0,0.0,0.0,11.019284,11.019284,5.509642,1163.911846


**Retrieve Defensive Stats**

Retrieves defensive data for each defending player

In [23]:
def_df = pd.read_csv('./stats/2021-22_def_stats.csv')

# Get def stats only from selected defenders
def_df = def_df[def_df['PLAYER_ID'].isin(off_df['DEF_PLAYER_ID'].tolist())]

print(def_df.shape)
def_df.head()

(21, 18)


,PLAYER_ID,PLAYER_NAME,PLAYER_POSITION,AGE,GP,G,FREQ,D_FGM,D_FGA,D_FG_PCT,NORMAL_FG_PCT,PCT_PLUSMINUS,W,L,MIN,STL,BLK,DREB
28,202340,Avery Bradley,G,31.0,41,41,1.0,4.29,8.24,0.521,0.448,0.073,15.0,26.0,29.0,0.7,1.7,5.0
92,1629012,Collin Sexton,G,23.0,11,11,1.0,4.09,9.27,0.441,0.448,-0.007,14.0,30.0,33.5,0.7,0.2,4.8
119,1630558,Davion Mitchell,G,23.0,43,43,1.0,4.05,9.74,0.415,0.441,-0.026,27.0,17.0,24.6,0.6,0.1,2.8
123,1628368,De'Aaron Fox,G,24.0,42,42,1.0,6.67,13.10,0.509,0.448,0.061,17.0,32.0,29.0,0.9,0.3,3.3
125,1629001,De'Anthony Melton,G,23.0,41,41,1.0,4.56,9.07,0.503,0.446,0.057,19.0,14.0,12.9,0.3,0.1,1.1


**Keras Model Implementation**

Use Keras Model from Tensorflow to predict offensive player's stats.
If there is an error, make sure to first install tensorflow. This can be done through Anaconda, or through the command 

```
!pip install tensorflow
```

In [293]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import numpy as np

In [294]:
# Clean data to use on model
X = def_df[def_df.columns[7:]].drop(columns=['W', 'L', 'MIN'])
y = off_df[off_df.columns[8:12]]
X = X.fillna(0)
y = y.fillna(0)

y = y.to_numpy()

# Generate training and verification data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [295]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

y_train.shape, X_train.shape

((16, 4), (16, 8))

In [296]:
# Define inputs, outputs, and layers
# TODO: Research layers shape and activation functions
inputs = keras.Input(shape=(X.columns.size,), dtype='float64')

x = tf.keras.layers.Dense(8, activation='relu')(inputs)

outputs = layers.Dense(y.shape[1], activation='relu')(x)

# Create and compile model
# TODO: Research optimizer and loss
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy')

model.summary()

Model: "model_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_38 (InputLayer)       [(None, 8)]               0         
                                                                 
 dense_82 (Dense)            (None, 8)                 72        
                                                                 
 dense_83 (Dense)            (None, 4)                 36        
                                                                 
Total params: 108
Trainable params: 108
Non-trainable params: 0
_________________________________________________________________


In [297]:
# Fit model
model.fit(X_train, y_train, epochs=500, batch_size=1)

Epoch 1/500
16/16 [==============================] - 1s 2ms/step - loss: 322.1024
Epoch 2/500
16/16 [==============================] - 0s 3ms/step - loss: 315.4373
Epoch 3/500
16/16 [==============================] - 0s 2ms/step - loss: 303.2061
Epoch 4/500
16/16 [==============================] - 0s 2ms/step - loss: 301.4273
Epoch 5/500
16/16 [==============================] - 0s 2ms/step - loss: 290.3911
Epoch 6/500
16/16 [==============================] - 0s 2ms/step - loss: 274.2128
Epoch 7/500
16/16 [==============================] - 0s 2ms/step - loss: 273.3107
Epoch 8/500
16/16 [==============================] - 0s 2ms/step - loss: 272.2512
Epoch 9/500
16/16 [==============================] - 0s 2ms/step - loss: 271.7035
Epoch 10/500
16/16 [==============================] - 0s 2ms/step - loss: 271.1294
Epoch 11/500
16/16 [==============================] - 0s 3ms/step - loss: 270.5086
Epoch 12/500
16/16 [==============================] - 0s 5ms/step - loss: 269.7645
Epoch 13/500


16/16 [==============================] - 0s 3ms/step - loss: 266.0815
Epoch 100/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0798
Epoch 101/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0782
Epoch 102/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0768
Epoch 103/500
16/16 [==============================] - 0s 4ms/step - loss: 266.0760
Epoch 104/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0751
Epoch 105/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0768
Epoch 106/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0781
Epoch 107/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0757
Epoch 108/500
16/16 [==============================] - 0s 2ms/step - loss: 266.0742
Epoch 109/500
16/16 [==============================] - 0s 2ms/step - loss: 266.0728
Epoch 110/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0718
Epoch 

16/16 [==============================] - 0s 3ms/step - loss: 266.0612
Epoch 197/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0612
Epoch 198/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0612
Epoch 199/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0612
Epoch 200/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0612
Epoch 201/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0611
Epoch 202/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0612
Epoch 203/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0612
Epoch 204/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0612
Epoch 205/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0612
Epoch 206/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0613
Epoch 207/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0614
Epoch 

16/16 [==============================] - 0s 2ms/step - loss: 266.0618
Epoch 294/500
16/16 [==============================] - 0s 2ms/step - loss: 266.0618
Epoch 295/500
16/16 [==============================] - 0s 2ms/step - loss: 266.0616
Epoch 296/500
16/16 [==============================] - 0s 2ms/step - loss: 266.0617
Epoch 297/500
16/16 [==============================] - 0s 2ms/step - loss: 266.0616
Epoch 298/500
16/16 [==============================] - 0s 2ms/step - loss: 266.0616
Epoch 299/500
16/16 [==============================] - 0s 2ms/step - loss: 266.0616
Epoch 300/500
16/16 [==============================] - 0s 2ms/step - loss: 266.0616
Epoch 301/500
16/16 [==============================] - 0s 2ms/step - loss: 266.0615
Epoch 302/500
16/16 [==============================] - 0s 2ms/step - loss: 266.0615
Epoch 303/500
16/16 [==============================] - 0s 2ms/step - loss: 266.0615
Epoch 304/500
16/16 [==============================] - 0s 2ms/step - loss: 266.0615
Epoch 

16/16 [==============================] - 0s 4ms/step - loss: 266.0611
Epoch 391/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0611
Epoch 392/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0611
Epoch 393/500
16/16 [==============================] - 0s 4ms/step - loss: 266.0611
Epoch 394/500
16/16 [==============================] - 0s 4ms/step - loss: 266.0611
Epoch 395/500
16/16 [==============================] - 0s 4ms/step - loss: 266.0611
Epoch 396/500
16/16 [==============================] - 0s 4ms/step - loss: 266.0611
Epoch 397/500
16/16 [==============================] - 0s 4ms/step - loss: 266.0611
Epoch 398/500
16/16 [==============================] - 0s 4ms/step - loss: 266.0611
Epoch 399/500
16/16 [==============================] - 0s 4ms/step - loss: 266.0612
Epoch 400/500
16/16 [==============================] - 0s 4ms/step - loss: 266.0612
Epoch 401/500
16/16 [==============================] - 0s 4ms/step - loss: 266.0613
Epoch 

16/16 [==============================] - 0s 4ms/step - loss: 266.0611
Epoch 488/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0611
Epoch 489/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0611
Epoch 490/500
16/16 [==============================] - 0s 4ms/step - loss: 266.0611
Epoch 491/500
16/16 [==============================] - 0s 4ms/step - loss: 266.0611
Epoch 492/500
16/16 [==============================] - 0s 4ms/step - loss: 266.0611
Epoch 493/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0611
Epoch 494/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0611
Epoch 495/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0611
Epoch 496/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0611
Epoch 497/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0611
Epoch 498/500
16/16 [==============================] - 0s 3ms/step - loss: 266.0612
Epoch 

In [298]:
y_pred = model.predict(X_test, batch_size=1)
score = model.evaluate(X_test, y_test, verbose=1)
print(score)

1/1 [==============================] - 0s 116ms/step - loss: 153.2358
153.2357940673828
